In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from recbole.quick_start import run_recbole
import pandas as pd
from recbole.quick_start import load_data_and_model
import os



In [9]:
run_recbole(config_file_list=['lastfm-360k-base.yaml'])

09 Jun 10:27    INFO  ['/Users/huangjiaqing/Desktop/Recommender Systems/RScode/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '-f', '/Users/huangjiaqing/Library/Jupyter/runtime/kernel-aa2ec524-86a1-4d8b-a722-43f3438c4bcf.json']
09 Jun 10:27    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = datasets/lastfm-360k
checkpoint_dir = ../checkpoint_saved/lastfm-360k/
show_progress = False
save_dataset = True
dataset_save_path = None
save_dataloaders = True
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 100
train_batch_size = 1024
learner = adam
learning_rate = 0.0005
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group

09 Jun 10:27    INFO  epoch 0 training [time: 2.52s, train loss: 241.4394]
09 Jun 10:27    INFO  epoch 0 evaluating [time: 10.21s, valid_score: 0.030700]
09 Jun 10:27    INFO  valid result: 
precision@10 : 0.0166    recall@10 : 0.0377    hit@10 : 0.1525    ndcg@10 : 0.0307    itemcoverage@10 : 0.0705    averagepopularity@10 : 1165.5967    shannonentropy@10 : 0.0013    giniindex@10 : 0.9974    tailpercentage@10 : 0.004
09 Jun 10:27    INFO  Saving current: ../checkpoint_saved/lastfm-360k/BPR-Jun-09-2025_10-27-13.pth
09 Jun 10:27    INFO  epoch 1 training [time: 2.39s, train loss: 203.9165]
09 Jun 10:27    INFO  epoch 1 evaluating [time: 10.53s, valid_score: 0.029300]
09 Jun 10:27    INFO  valid result: 
precision@10 : 0.0156    recall@10 : 0.0352    hit@10 : 0.1424    ndcg@10 : 0.0293    itemcoverage@10 : 0.0136    averagepopularity@10 : 1239.4749    shannonentropy@10 : 0.0051    giniindex@10 : 0.9996    tailpercentage@10 : 0.0009
09 Jun 10:27    INFO  epoch 2 training [time: 2.43s, tra

{'best_valid_score': 0.0621,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('precision@10', 0.0344),
              ('recall@10', 0.0784),
              ('hit@10', 0.2987),
              ('ndcg@10', 0.0621),
              ('itemcoverage@10', 0.1021),
              ('averagepopularity@10', 631.3722),
              ('shannonentropy@10', 0.0016),
              ('giniindex@10', 0.9898),
              ('tailpercentage@10', 0.0041)]),
 'test_result': OrderedDict([('precision@10', 0.0332),
              ('recall@10', 0.0754),
              ('hit@10', 0.2904),
              ('ndcg@10', 0.0595),
              ('itemcoverage@10', 0.1021),
              ('averagepopularity@10', 631.3722),
              ('shannonentropy@10', 0.0016),
              ('giniindex@10', 0.9898),
              ('tailpercentage@10', 0.0041)])}

In [10]:
def save_interaction_for_training(inter_feat, dataset, save_path):
    df = {}
    for field in inter_feat.interaction.keys():
        if field == dataset.uid_field:
            df['user_id:token'] = dataset.id2token(field, inter_feat[field].numpy().tolist())
        elif field == dataset.iid_field:
            df['artist_id:token'] = dataset.id2token(field, inter_feat[field].numpy().tolist())
        elif field == config['LABEL_FIELD']:
            df['label:float'] = inter_feat[field].numpy()
        else:
            df[field] = inter_feat[field].numpy()
    pd.DataFrame(df).to_csv(save_path, sep='\t', index=False)

# 加载模型与数据
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='../checkpoint_saved/lastfm-360k/BPR-Jun-09-2025_00-53-20.pth'
)

# 获取 interaction 对象
train_inter = train_data.dataset.inter_feat
valid_inter = valid_data.dataset.inter_feat
test_inter  = test_data.dataset.inter_feat

# 保存路径
save_dir = '../datasets/split_datasets/lastfm-360k'
os.makedirs(save_dir, exist_ok=True)

# 保存为 .inter 文件
save_interaction_for_training(train_inter, dataset, os.path.join(save_dir, 'lastfm-360k.train.inter'))
save_interaction_for_training(valid_inter, dataset, os.path.join(save_dir, 'lastfm-360k.valid.inter'))
save_interaction_for_training(test_inter, dataset, os.path.join(save_dir, 'lastfm-360k.test.inter'))
print("Train, valid, test datasets saved successfully.")

09 Jun 10:55    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = datasets/lastfm-360k
checkpoint_dir = ../checkpoint_saved/lastfm-360k/
show_progress = True
save_dataset = True
dataset_save_path = None
save_dataloaders = True
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 100
train_batch_size = 1024
learner = adam
learning_rate = 0.0005
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Precision', 'Recall', 'Hit', 'NDCG', 'ItemCoverage', 'AveragePopularity', 'ShannonEntropy', 'GiniIndex', 'TailPercentage']
topk = [10]
valid_metric = NDCG

Train, valid, test datasets saved successfully.


In [11]:
from recbole.quick_start import load_data_and_model
from recbole.utils.case_study import full_sort_topk
from tqdm import tqdm
import pandas as pd

# 1. 加载模型和数据
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='../checkpoint_saved/lastfm-360k/BPR-Jun-09-2025_00-53-20.pth'
)

# Step 1: Get all internal user IDs
all_uids = list(range(dataset.user_num))

# Step 2: Filter out users who have no interactions in the test set
valid_uids = [uid for uid in tqdm(all_uids) if test_data.uid2history_item[uid] is not None]

# Step 3: Convert to Series
import numpy as np
uid_series = np.array(valid_uids)

# Step 4: Run full_sort_topk
topk_scores, topk_index = full_sort_topk(uid_series, model, test_data, k=30, device=config['device'])

# Step 5: Convert internal item IDs to external tokens
external_item_lists = [dataset.id2token(dataset.iid_field, row.cpu().tolist()) for row in topk_index]
external_user_list = [dataset.id2token(dataset.uid_field, [uid])[0] for uid in uid_series]

# Step 6: Save as DataFrame
df = pd.DataFrame({
    'user_id': external_user_list,
    'topk_items': [','.join(items) for items in external_item_lists]
})
display(df.head())
df.to_csv('outputs/lastfm_all_user_top30.csv', index=False)
print("save lastfm_all_user_top30 successfully")

09 Jun 10:55    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = datasets/lastfm-360k
checkpoint_dir = ../checkpoint_saved/lastfm-360k/
show_progress = True
save_dataset = True
dataset_save_path = None
save_dataloaders = True
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 100
train_batch_size = 1024
learner = adam
learning_rate = 0.0005
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Precision', 'Recall', 'Hit', 'NDCG', 'ItemCoverage', 'AveragePopularity', 'ShannonEntropy', 'GiniIndex', 'TailPercentage']
topk = [10]
valid_metric = NDCG

,user_id,topk_items
0,0004de6c3c32daa599bd03a37ce2356d5e7cc23f,"31745282-b1ea-4d62-939f-226b14d68e7c,7fa7fc04-..."
1,0007e26aafcfc0b6dcb87d7041583fbb7cced88a,"cc197bad-dc9c-440d-a5b5-d52ba2e14234,b10bbbfc-..."
2,0009f70b52e48eca23a446d8a6f8bd2663691c54,"14b22b4b-06d5-4b82-8284-29d29b58945f,a505bb48-..."
3,001078f2f557a1afd9b9618144fc0b442481fd13,"cc197bad-dc9c-440d-a5b5-d52ba2e14234,b10bbbfc-..."
4,00292157f9c4e4b11464e886f27fff42f1ff442d,"9a709693-b4f8-4da9-8cc1-038c911a61be,14b22b4b-..."


save lastfm_all_user_top30 successfully


In [14]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Step 1: 加载推荐结果 & 用户性别
topk_df = pd.read_csv('outputs/lastfm_all_user_top30.csv')
user_df = pd.read_csv('../datasets/nl_gender_norm/lastfm-360k/lastfm-360k.user', sep='\t')
user2gender = dict(zip(user_df['user_id:token'], user_df['gender:token']))

# Step 2: 加载测试集 ground truth（正反馈）
test_df = pd.read_csv('../datasets/split_datasets/lastfm-360k/lastfm-360k.test.inter', sep='\t')
test_df = test_df[test_df['label:float'] == 1.0]
user2ground_truth = test_df.groupby('user_id:token')['artist_id:token'].agg(set).to_dict()

# Step 3: NDCG@10 计算函数
def ndcg_at_k(preds, true_items, k=10):
    dcg = 0.0
    for i, item in enumerate(preds[:k]):
        if item in true_items:
            dcg += 1.0 / np.log2(i + 2)
    ideal_len = min(len(true_items), k)
    idcg = sum(1.0 / np.log2(i + 2) for i in range(ideal_len))
    return dcg / idcg if idcg > 0 else 0.0

# Step 4: 分组统计
ndcg_male, ndcg_female, ndcg_all = [], [], []

for _, row in topk_df.iterrows():
    uid = row['user_id']
    pred_items = row['topk_items'].split(',') if isinstance(row['topk_items'], str) else row['topk_items']
    true_items = user2ground_truth.get(uid, set())
    gender = user2gender.get(uid, None)

    if gender not in ('m', 'f') or not true_items:
        continue

    ndcg = ndcg_at_k(pred_items, true_items, k=10)
    if gender == 'm':
        ndcg_male.append(ndcg)
        ndcg_all.append(ndcg)
    else:
        ndcg_female.append(ndcg)
        ndcg_all.append(ndcg)

# Step 5: 输出
print(f'NDCG@10 (All):    {np.mean(ndcg_all):.4f} over {len(ndcg_all)} users')
print(f'NDCG@10 (Male):   {np.mean(ndcg_male):.4f} over {len(ndcg_male)} users')
print(f'NDCG@10 (Female): {np.mean(ndcg_female):.4f} over {len(ndcg_female)} users')

NDCG@10 (All):    0.0594 over 8769 users
NDCG@10 (Male):   0.0601 over 7044 users
NDCG@10 (Female): 0.0565 over 1725 users
